# **INTRODUCTION**

* * *

This workflow consists of several processes used for creating the so called 'berggrafiek' on the dashboard. 

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Views](#views)**
3. **[Stored Procedures](#stored-procedures)**
4. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**

---

### STAGING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSSTAGE_VACC_DELIVER_ADMIN'))
CREATE SEQUENCE [dbo].[SEQ_VWSSTAGE_VACC_DELIVER_ADMIN]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSSTAGE.VWS_VACCINE_DELIVERIES_ADMINISTERED'))
CREATE TABLE [VWSSTAGE].[VWS_VACCINE_DELIVERIES_ADMINISTERED] (
    [ID]                 INT           DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_VACC_DELIVER_ADMIN]) NOT NULL,
    [DATE_OF_REPORT]     VARCHAR (100) NULL,
    [DATE_FIRST_DAY]     VARCHAR (100) NULL,
    [VALUE_TYPE]         VARCHAR (100) NULL,
    [VALUE_NAME]         VARCHAR (100) NULL,
    [REPORT_STATUS]      VARCHAR (100) NULL,
    [VALUE]              VARCHAR (100) NULL,
    [DATE_LAST_INSERTED] DATETIME      DEFAULT (getdate()) NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 3) CREATE STAGE INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_VWS_VACCINE_DELIVERIES_ADMINISTERED_STAGE')
CREATE NONCLUSTERED INDEX [IX_VWS_VACCINE_DELIVERIES_ADMINISTERED_STAGE]
    ON [VWSSTAGE].[VWS_VACCINE_DELIVERIES_ADMINISTERED]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [DATE_FIRST_DAY], [VALUE_TYPE], [VALUE_NAME], [REPORT_STATUS], [VALUE]);
GO

### INTERMEDIATE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE INTER SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSINTER_VACC_DELIVER_ADMIN'))
CREATE SEQUENCE [dbo].[SEQ_VWSINTER_VACC_DELIVER_ADMIN]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1; 
GO

-- 2) CREATE INTER TABLE(S).....

IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSINTER.VWS_VACCINE_DELIVERIES_ADMINISTERED'))
CREATE TABLE [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED] (
    [ID]                 INT           DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSINTER_VACC_DELIVER_ADMIN]) NOT NULL,
    [DATE_OF_REPORT]     DATETIME      NULL,
    [DATE_FIRST_DAY]     DATETIME      NULL,
    [VALUE_TYPE]         VARCHAR (100) NULL,
    [VALUE_NAME]         VARCHAR (100) NULL,
    [REPORT_STATUS]      VARCHAR (100) NULL,
    [VALUE]              BIGINT        NULL,
    [DATE_LAST_INSERTED] DATETIME      DEFAULT (getdate()) NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 3) CREATE INTER INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_VWS_VACCINE_DELIVERIES_ADMINISTERED_INTER')
CREATE NONCLUSTERED INDEX [IX_VWS_VACCINE_DELIVERIES_ADMINISTERED_INTER]
    ON [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_OF_REPORT], [DATE_FIRST_DAY], [VALUE_TYPE], [VALUE_NAME], [REPORT_STATUS], [VALUE]);
GO

### DESTINATIONS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE VWSDEST SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSDEST_VACCINE_ADMINISTERED'))
CREATE SEQUENCE [dbo].[SEQ_VWSDEST_VACCINE_ADMINISTERED]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1; 
GO

-- 2) CREATE VWSDEST TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSDEST.VWS_VACCINE_ADMINISTERED'))
BEGIN
    CREATE TABLE [VWSDEST].[VWS_VACCINE_ADMINISTERED] (
        [ID]                 INT           DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSDEST_VACCINE_ADMINISTERED]) NOT NULL,
        [DATE_FIRST_DAY]     DATETIME      NULL,
        [DATE_START_UNIX]    BIGINT        NULL,
        [DATE_END_UNIX]      BIGINT        NULL,
        [VALUE_TYPE]         VARCHAR (100) NULL,
        [REPORT_STATUS]      VARCHAR (100) NULL,
        [AstraZeneca]        BIGINT        NULL,
        [BioNTech/Pfizer]    BIGINT        NULL,
        [CureVac]            BIGINT        NULL,
        [Janssen]            BIGINT        NULL,
        [Moderna]            BIGINT        NULL,
        [Sanofi]             BIGINT        NULL,
        [Novavax]             BIGINT        NULL,
        [DATE_LAST_INSERTED] DATETIME      DEFAULT (getdate()) NULL,
        PRIMARY KEY CLUSTERED ([ID] ASC)
    );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
    WHERE [NAME] IN (N'Novavax')
    AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[VWS_VACCINE_ADMINISTERED]'))
        BEGIN
            ALTER TABLE [VWSDEST].[VWS_VACCINE_ADMINISTERED] 
            ADD 
                [Novavax] BIGINT NULL;
	    END
END
GO

-- 3) CREATE VWSDEST INDEX(ES).....
DROP INDEX IF EXISTS IX_DEST_VWS_VACCINE_ADMINISTERED ON [VWSDEST].[VWS_VACCINE_ADMINISTERED]

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_VWS_VACCINE_ADMINISTERED_2')
CREATE NONCLUSTERED INDEX [IX_DEST_VWS_VACCINE_ADMINISTERED_2]
    ON [VWSDEST].[VWS_VACCINE_ADMINISTERED]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_FIRST_DAY], [DATE_START_UNIX], [DATE_END_UNIX], [VALUE_TYPE], [REPORT_STATUS], [AstraZeneca], [BioNTech/Pfizer], [CureVac], [Janssen], [Moderna], [Sanofi], [Novavax] );


GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE VWSDEST SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSDEST_VACCINE_DELIVERIES'))
CREATE SEQUENCE [dbo].[SEQ_VWSDEST_VACCINE_DELIVERIES]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1;


GO

-- 2) CREATE VWSDEST TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSDEST.VWS_VACCINE_DELIVERIES'))
CREATE TABLE [VWSDEST].[VWS_VACCINE_DELIVERIES] (
    [ID]                  INT           DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSDEST_VACCINE_DELIVERIES]) NOT NULL,
    [DATE_FIRST_DAY]      DATETIME      NULL,
    [DATE_START_UNIX]     BIGINT        NULL,
    [DATE_END_UNIX]       BIGINT        NULL,
    [VALUE_TYPE]          VARCHAR (100) NULL,
    [REPORT_STATUS]       VARCHAR (100) NULL,
    [VALUE]               BIGINT        NULL,
    [DATE_LAST_INSERTED]  DATETIME      DEFAULT (getdate()) NULL,
    [DATE_OF_REPORT_UNIX] BIGINT        NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 3) CREATE VWSDEST INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_VWS_VACCINE_DELIVERIES')
CREATE NONCLUSTERED INDEX [IX_DEST_VWS_VACCINE_DELIVERIES]
    ON [VWSDEST].[VWS_VACCINE_DELIVERIES]([DATE_LAST_INSERTED] ASC)
    INCLUDE([DATE_FIRST_DAY], [DATE_START_UNIX], [DATE_END_UNIX], [VALUE_TYPE], [REPORT_STATUS], [VALUE]);

GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE VWSDEST SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSDEST_VACCINE_DELIVERIES_WEEKLY'))
CREATE SEQUENCE [dbo].[SEQ_VWSDEST_VACCINE_DELIVERIES_WEEKLY]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE VWSDEST TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSDEST.VWS_VACCINE_DELIVERIES_WEEKLY'))
BEGIN
    CREATE TABLE [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY] (
        [ID]                 INT           DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSDEST_VACCINE_DELIVERIES_WEEKLY]) NOT NULL,
        [DATE_FIRST_DAY]     DATETIME      NULL,
        [DATE_OF_REPORT]     DATETIME      NULL,
        [DATE_START_UNIX]    BIGINT        NULL,
        [DATE_END_UNIX]      BIGINT        NULL,
        [VALUE_TYPE]         VARCHAR (100) NULL,
        [REPORT_STATUS]      VARCHAR (100) NULL,
        [AstraZeneca]        BIGINT        NULL,
        [BioNTech/Pfizer]    BIGINT        NULL,
        [CureVac]            BIGINT        NULL,
        [Janssen]            BIGINT        NULL,
        [Moderna]            BIGINT        NULL,
        [Sanofi]             BIGINT        NULL,
       -- [Novavax]             BIGINT        NULL,
        [TOTAL_VALUE]        BIGINT        NULL,
        [DATE_LAST_INSERTED] DATETIME      DEFAULT (getdate()) NULL,
        PRIMARY KEY CLUSTERED ([ID] ASC)
    );
END
--ELSE
--BEGIN
--    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
--    WHERE [NAME] IN (N'Novavax')
--    AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY]'))
--        BEGIN
--            ALTER TABLE [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY] 
--            ADD 
--                [Novavax]             BIGINT        NULL;
--	    END
--END
GO

-- 3) CREATE VWSDEST INDEX(ES).....
--DROP INDEX IF EXISTS IX_DEST_VWS_VACCINE_DELIVERIES_WEEKLY ON [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY]

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_VWS_VACCINE_DELIVERIES_WEEKLY')
CREATE NONCLUSTERED INDEX [IX_DEST_VWS_VACCINE_DELIVERIES_WEEKLY]
    ON [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY]([DATE_LAST_INSERTED] ASC, [VALUE_TYPE] ASC, [REPORT_STATUS] ASC)
    INCLUDE([DATE_FIRST_DAY], [DATE_OF_REPORT], [DATE_START_UNIX], [DATE_END_UNIX], [AstraZeneca], [BioNTech/Pfizer], [CureVac], [Janssen], [Moderna], [Sanofi], [TOTAL_VALUE]);

GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE VWSDEST SEQUENCE(S).....
IF NOT EXISTS (SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID('DBO.SEQ_VWSDEST_VWS_VACCINE_PLANNED'))
CREATE SEQUENCE [dbo].[SEQ_VWSDEST_VWS_VACCINE_PLANNED]
    AS BIGINT
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE VWSDEST TABLE(S).....
IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE object_id = OBJECT_ID('VWSDEST.VWS_VACCINE_PLANNED'))
CREATE TABLE [VWSDEST].[VWS_VACCINE_PLANNED] (
    [ID]                 INT      DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_VWS_VACCINE_PLANNED]) NOT NULL,
    [DOSES]              BIGINT   NOT NULL,
    [DATE_START_UNIX]    BIGINT   NOT NULL,
    [DATE_END_UNIX]      BIGINT   NOT NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (getdate()) NOT NULL
);
GO

-- 3) CREATE VWSDEST INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_VWS_VACCINE_PLANNED')
CREATE CLUSTERED INDEX [IX_VWS_VACCINE_PLANNED]
    ON [VWSDEST].[VWS_VACCINE_PLANNED]([DATE_LAST_INSERTED] DESC);

GO

# **VIEWS**

---

In [ ]:
CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_ADMINISTERED AS
SELECT 
     [DATE_START_UNIX]
    ,[DATE_END_UNIX]
    ,[AstraZeneca]     AS [astra_zeneca]
    ,[BioNTech/Pfizer] AS [pfizer]
--    ,[CureVac]         AS [cure_vac]
    ,[Janssen]
    ,[Moderna]
    ,[Novavax]
--    ,[Sanofi]
    ,[AstraZeneca] + [BioNTech/Pfizer] + [Moderna]  + [Janssen] + [Novavax]   AS [total]
    ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)   AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.VWS_VACCINE_ADMINISTERED 
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_ADMINISTERED])
  AND REPORT_STATUS = 'reported'

In [ ]:
CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_ADMINISTERED_EST AS
    SELECT 
        [DATE_START_UNIX]
        ,[DATE_END_UNIX]
        ,[AstraZeneca]                                  AS [astra_zeneca]
        ,[BioNTech/Pfizer]                              AS [pfizer]
    --    ,[CureVac]         AS [cure_vac]
        ,[Janssen]
        ,[Moderna]
    --    ,[Novavax]
    --    ,[Sanofi]
        ,[AstraZeneca] + [BioNTech/Pfizer] + [Moderna] +  [Janssen]  AS [total]
        ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) 
                                                        AS DATE_OF_INSERTION_UNIX
    FROM 
        VWSDEST.VWS_VACCINE_ADMINISTERED 
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_ADMINISTERED])
    AND REPORT_STATUS = 'estimated'

In [ ]:
CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_DELIVERIES AS
SELECT 
     [VALUE] AS TOTAL
    ,DATE_START_UNIX
    ,DATE_OF_REPORT_UNIX
    ,DATE_END_UNIX
    ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.VWS_VACCINE_DELIVERIES 
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_DELIVERIES])
  AND REPORT_STATUS = 'reported'

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_DELIVERIES_EST AS
SELECT 
     [VALUE] AS TOTAL
    ,DATE_START_UNIX
    ,DATE_END_UNIX
    ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.VWS_VACCINE_DELIVERIES 
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_DELIVERIES])
  AND REPORT_STATUS = 'estimated'

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_DELIVERIES_WEEKLY AS
SELECT 
     [AstraZeneca]     AS [astra_zeneca]
    ,[BioNTech/Pfizer] AS [bio_n_tech_pfizer]
--      ,[CureVac]         AS [cure_vac]
      ,[Janssen]
      ,[Moderna]
--      ,[Novavax]
--      ,[Sanofi]
      ,[TOTAL_VALUE] as [total]
      ,IIF([REPORT_STATUS] = 'estimated','true','false') AS is_estimate
      ,[DATE_START_UNIX]
      ,[DATE_END_UNIX]
      ,dbo.CONVERT_DATETIME_TO_ISO_WEEKNUMBER([DATE_FIRST_DAY]) as WEEK_NUMBER
    --   ,ROW_NUMBER() OVER (ORDER BY DATE_START_UNIX) as WEEK_NUMBER
      ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_OF_REPORT) AS DATE_OF_REPORT_UNIX
      ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
  FROM [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY])

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE  OR ALTER VIEW VWSDEST.V_VWS_VACCINE_DELIVERIES_WEEKLY_TIMESPAN AS
SELECT 
     SUM(TOTAL_VALUE)          AS doses
    ,COUNT(*)                  AS time_span_weeks
    ,MIN([DATE_START_UNIX])    AS [DATE_START_UNIX]
    ,MAX([DATE_END_UNIX])      AS [DATE_END_UNIX]
    ,MAX(dbo.CONVERT_DATETIME_TO_UNIX([DATE_OF_REPORT]))          AS [DATE_OF_REPORT_UNIX]
    ,MAX(dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]))      AS [DATE_OF_INSERTION_UNIX]

  FROM [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX ([DATE_LAST_INSERTED]) FROM [VWSDEST].[VWS_VACCINE_DELIVERIES_WEEKLY])
        AND [REPORT_STATUS] = 'estimated'

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW VWSDEST.V_VWS_VACCINE_ADMINISTERED_PLANNED_TEMP
AS
	SELECT 
		DOSES,
		DATE_START_UNIX,
		DATE_END_UNIX,
		dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED])			AS [DATE_OF_INSERTION_UNIX]
	FROM VWSDEST.VWS_VACCINE_PLANNED
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.VWS_VACCINE_PLANNED)
GO


In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW VWSREPORT.V_PBI_VWS_VACCINE_ADMINISTERED_PLANNED_TEMP
AS
	SELECT 
		DOSES,
		DATE_START_UNIX,
		DATE_END_UNIX,
		[DATE_LAST_INSERTED]			AS [DATE_OF_INSERTION]
	FROM VWSDEST.VWS_VACCINE_PLANNED
	WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.VWS_VACCINE_PLANNED)

# **STORED PROCEDURES**

---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

-- 1) CREATE STORE PROCEDURE(S) STAGE -> INTER.....
CREATE OR ALTER PROCEDURE [dbo].[SP_VWS_VACCINE_DELIVERIES_ADMINISTERED_INTER]
AS
BEGIN
    INSERT INTO VWSINTER.VWS_VACCINE_DELIVERIES_ADMINISTERED (
    DATE_OF_REPORT ,
    DATE_FIRST_DAY ,
    VALUE_TYPE ,
    VALUE_NAME ,
    REPORT_STATUS,
    [VALUE] 
    )
    SELECT
        -- Datetime format in source file is dd-mm-yyyy

    CONVERT(DATETIME,DATE_OF_REPORT, 105) AS DATE_OF_REPORT,
    CONVERT(DATETIME,DATE_FIRST_DAY, 105) AS DATE_FIRST_DAY,
    VALUE_TYPE ,
    VALUE_NAME ,
    REPORT_STATUS,
    CAST([VALUE] AS BIGINT) AS [VALUE]

    FROM 
        VWSSTAGE.VWS_VACCINE_DELIVERIES_ADMINISTERED
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) 
                                  FROM VWSSTAGE.VWS_VACCINE_DELIVERIES_ADMINISTERED)
END;
GO

### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.
    
-- 1) CREATE STORE PROCEDURE(S) INTER -> DEST.....
CREATE OR ALTER PROCEDURE [dbo].[SP_VWS_VACCINE_ADMINISTERED_DEST]
AS
BEGIN
WITH BASE_CTE AS
(
SELECT [DATE_FIRST_DAY]
      ,[VALUE_TYPE]
      ,[VALUE_NAME]
      ,[REPORT_STATUS]
      ,[VALUE]
  FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]
  WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]) 
    AND VALUE_TYPE = 'vaccine_administered' 
)
,
PIVOT_CTE AS
(
    SELECT 
         [AstraZeneca], [BioNTech/Pfizer], [CureVac], [Janssen], [Moderna], [Sanofi], [Novavax] 
        ,[DATE_FIRST_DAY]
        ,[VALUE_TYPE]
        ,[REPORT_STATUS]
    FROM   BASE_CTE
    PIVOT  
    (  
        SUM ([VALUE])  
        FOR VALUE_NAME IN  
            ( [AstraZeneca],[BioNTech/Pfizer],[CureVac],[Janssen],[Moderna],[Sanofi], [Novavax] 
            )  
    ) AS pvt  
)
INSERT INTO VWSDEST.VWS_VACCINE_ADMINISTERED
(
   [AstraZeneca]      
  ,[BioNTech/Pfizer]  
  ,[CureVac]          
  ,[Janssen]          
  ,[Moderna]          
  ,[Sanofi]           
  ,[Novavax]
  ,[DATE_FIRST_DAY]  
  ,DATE_START_UNIX    
  ,DATE_END_UNIX      
  ,[VALUE_TYPE]      
  ,[REPORT_STATUS]   
)
SELECT  
         ISNULL([AstraZeneca]        ,0) AS [AstraZeneca]    
        ,ISNULL([BioNTech/Pfizer]    ,0) AS [BioNTech/Pfizer]
        ,ISNULL([CureVac]            ,0) AS [CureVac]        
        ,ISNULL([Janssen]            ,0) AS [Janssen]        
        ,ISNULL([Moderna]            ,0) AS [Moderna]        
        ,ISNULL([Sanofi]             ,0) AS [Sanofi]   
        ,ISNULL([Novavax]            ,0) AS [Novavax]
        ,[DATE_FIRST_DAY]
        ,[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_FIRST_DAY])                 AS DATE_START_UNIX
        ,[dbo].[CONVERT_DATETIME_TO_UNIX](DATEADD(DAY,6,[DATE_FIRST_DAY]))  AS DATE_END_UNIX
        ,[VALUE_TYPE]
        ,[REPORT_STATUS]
FROM PIVOT_CTE
  ORDER BY DATE_FIRST_DAY
END;
GO

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_VWS_VACCINE_DELIVERIES_DEST]
AS
BEGIN

INSERT INTO VWSDEST.VWS_VACCINE_DELIVERIES
(
    [DATE_OF_REPORT_UNIX]
    ,[DATE_FIRST_DAY]
    ,DATE_START_UNIX
    ,DATE_END_UNIX
    ,[VALUE_TYPE]
    ,[REPORT_STATUS]
    ,[VALUE]
)
SELECT 
       [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_OF_REPORT) AS DATE_OF_REPORT_UNIX
      ,[DATE_FIRST_DAY]
      ,[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_FIRST_DAY])                 AS DATE_START_UNIX
      ,[dbo].[CONVERT_DATETIME_TO_UNIX](DATEADD(DAY,6,[DATE_FIRST_DAY]))  AS DATE_END_UNIX
      ,[VALUE_TYPE]
      ,[REPORT_STATUS]
      ,[VALUE]
  FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]
  WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]) 
    AND VALUE_TYPE = 'vaccine_delivery' 
    ORDER BY DATE_FIRST_DAY
END;
GO

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_VWS_VACCINE_DELIVERIES_WEEKLY]
AS
BEGIN
WITH BASE_CTE AS
(
SELECT [DATE_FIRST_DAY]
      ,DATE_OF_REPORT
      ,[VALUE_TYPE]
      ,[VALUE_NAME]
      ,[REPORT_STATUS]
      ,[VALUE]
  FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]
  WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[VWS_VACCINE_DELIVERIES_ADMINISTERED]) 
    AND VALUE_TYPE = 'vaccine_delivery_weekly' 
)
,
TOTAL_CTE AS (
SELECT [DATE_FIRST_DAY]
      ,[VALUE_TYPE]
      ,SUM([VALUE]) AS [TOTAL_VALUE]
  FROM [BASE_CTE]
    GROUP BY VALUE_TYPE, DATE_FIRST_DAY
)
,
PIVOT_CTE AS
(
    SELECT 
         [AstraZeneca], [BioNTech/Pfizer], [CureVac], [Janssen], [Moderna], [Sanofi] --, [Novavax]
        ,[DATE_FIRST_DAY]
        ,[DATE_OF_REPORT]
        ,[VALUE_TYPE]
        ,[REPORT_STATUS]
    FROM   BASE_CTE
    PIVOT  
    (  
        SUM ([VALUE])  
        FOR VALUE_NAME IN  
            ( [AstraZeneca],[BioNTech/Pfizer],[CureVac],[Janssen],[Moderna],[Sanofi] -- ,[Novavax]
            )  
    ) AS pvt  
)
INSERT INTO VWSDEST.VWS_VACCINE_DELIVERIES_WEEKLY
(
   [AstraZeneca]      
  ,[BioNTech/Pfizer]  
  ,[CureVac]          
  ,[Janssen]          
  ,[Moderna]          
  ,[Sanofi]     
 -- ,[Novavax]      
  ,TOTAL_VALUE
  ,[DATE_FIRST_DAY]  
  ,DATE_OF_REPORT
  ,DATE_START_UNIX    
  ,DATE_END_UNIX      
  ,[VALUE_TYPE]      
  ,[REPORT_STATUS]   
)

SELECT  
         ISNULL([AstraZeneca]        ,0) AS [AstraZeneca]    
        ,ISNULL([BioNTech/Pfizer]    ,0) AS [BioNTech/Pfizer]
        ,ISNULL([CureVac]            ,0) AS [CureVac]        
        ,ISNULL([Janssen]            ,0) AS [Janssen]        
        ,ISNULL([Moderna]            ,0) AS [Moderna]        
        ,ISNULL([Sanofi]             ,0) AS [Sanofi]   
    --    ,ISNULL([Novavax]            ,0) AS [Novavax]
        ,T2.TOTAL_VALUE
        ,T1.[DATE_FIRST_DAY]
        ,T1.DATE_OF_REPORT
        ,[dbo].[CONVERT_DATETIME_TO_UNIX](T1.[DATE_FIRST_DAY])                 AS DATE_START_UNIX
        ,[dbo].[CONVERT_DATETIME_TO_UNIX](DATEADD(DAY,6,T1.[DATE_FIRST_DAY]))  AS DATE_END_UNIX
        ,T1.[VALUE_TYPE]
        ,T1.[REPORT_STATUS]
FROM PIVOT_CTE T1
  LEFT JOIN TOTAL_CTE T2 ON T1.[DATE_FIRST_DAY] = T2.[DATE_FIRST_DAY] AND T1.[VALUE_TYPE] = T2.[VALUE_TYPE]
  ORDER BY DATE_FIRST_DAY

END;
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_VWS_VACCINE_PLANNED_DEST]
AS
BEGIN
WITH
		PlannedAdministered ([DATE_FIRST_DAY], [DATE_LAST_INSERTED], REPORT_STATUS, [VALUE])
		AS (
			SELECT
				 CAST(DATE_FIRST_DAY as date)															AS [DATE_FIRST_DAY]
				,CAST(DATE_LAST_INSERTED as date)														AS [DATE_LAST_INSERTED]
				,REPORT_STATUS
				,SUM([VALUE])																			AS [VALUE]
	
			FROM 
				VWSINTER.VWS_VACCINE_DELIVERIES_ADMINISTERED
			WHERE
				VALUE_TYPE = 'vaccine_administered' AND
				DATE_LAST_INSERTED = 
					(SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.VWS_VACCINE_DELIVERIES_ADMINISTERED)
			GROUP BY
				CAST(DATE_FIRST_DAY as date)
				,CAST(DATE_LAST_INSERTED as date)
				,REPORT_STATUS
		),
		PlannedAdministered2 ([DOSES],[DATE_START],[DATE_END],[DATE_LAST_INSERTED])
		AS (
			SELECT
				 [VALUE] - LAG([VALUE],1,0) OVER (ORDER BY DATE_FIRST_DAY ASC)							AS [DOSES]
				,DATE_FIRST_DAY																			AS [DATE_START]
				,DATEADD(day,-1,LEAD(DATE_FIRST_DAY,1,NULL) OVER (ORDER BY DATE_FIRST_DAY ASC))			AS [DATE_END]
				,DATE_LAST_INSERTED																		AS [DATE_LAST_INSERTED]
		
			FROM 
				PlannedAdministered
		)

        INSERT INTO VWSDEST.VWS_VACCINE_PLANNED
        (
            [DOSES]
            ,DATE_START_UNIX
            ,DATE_END_UNIX
        )
		SELECT 
			[DOSES]
			,dbo.CONVERT_DATETIME_TO_UNIX([DATE_START])			AS [DATE_START_UNIX]
			,dbo.CONVERT_DATETIME_TO_UNIX([DATE_END])			AS [DATE_END_UNIX]
		FROM
			PlannedAdministered2
		WHERE
			DATE_START <= CAST(GETDATE() as date) AND
			CAST(GETDATE() as date) <= DATE_END
END;
GO

# **DROP ARTIFACTS**

---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED',
        @proto_name VARCHAR(50) = 'NL';

-- 1) DROP PROTOS CONFIGURATION(S).....
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [VIEW_ID] IN (
    SELECT [ID] AS VID FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'
    ))
    AND [PROTO_ID] IN (
        SELECT [ID] FROM [DATATINO_PROTO_1].[PROTOS] protos 
        WHERE [NAME] = @proto_name
    ) AND [NAME] IN (
        'vaccine_administered_planned'
    );

DELETE FROM [DATATINO_PROTO_1].[VIEWS]
WHERE [ID] IN (
    SELECT 
        [ID] AS VID 
    FROM  [DATATINO_PROTO_1].[VIEWS]
    WHERE TRIM([NAME]) IN (
        'VWSDEST.V_MANUAL_INPUT_VWS_VACCINE_SHOTS_PLANNED'
    )
)

-- 2) DROP PROCESS(ES)......
;WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        CONCAT('PROCESS_', @workflow_name)
    ) AND v_workflows.[NAME] = @workflow_name
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);

-- 4) DROP SOURCE(S).....
DELETE FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] IN (
    CONCAT('SOURCE_', @workflow_name)
);

-- 5) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );

-- 6) DROP DATAFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[H_DATAFLOWS]
WHERE [NAME] IN (
    CONCAT('PROCESS_', @workflow_name),
    @workflow_name
);

DELETE  [DATATINO_ORCHESTRATOR_1].[DATAFLOWS]
WHERE [NAME] IN (
    CONCAT('PROCESS_', @workflow_name),
    @workflow_name
)
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'VWS_VACCINE_DELIVERIES_ADMINISTERED',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 12 * * MON-FRI', -- AT 02:00 PM, ON monday - friday
    @active = @is_active;

### Sources

#### Staging sources

In [ ]:



-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50),
        @delimiter_type VARCHAR(50),
        @source_type VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/VWS/vaccine_deliveries_administered_*.csv'
    WHEN 'acceptance' THEN 'datafiles/VWS/vaccine_deliveries_administered_*.csv'
    ELSE 'datafiles/VWS/vaccine_deliveries_administered_*.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'N/A'
    WHEN 'acceptance' THEN 'N/A'
    ELSE 'N/A'
END;

SET @delimiter_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'SemiColon'
    WHEN 'acceptance' THEN 'SemiColon'
    ELSE 'SemiColon'
END;

SET @source_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'CsvFile'
    WHEN 'acceptance' THEN 'CsvFile'
    ELSE 'CsvFile'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_VWS_Vaccination_Deliveries_Administered';
SET @source_description = 'Loads VWS Vaccination Deliveries Administered data into target table';
SET @target_name = 'VWSSTAGE.VWS_VACCINE_DELIVERIES_ADMINISTERED';


SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'date_of_report|date_first_day|value_type|value_name|report_status|value',
    @target_columns = 'DATE_OF_REPORT|DATE_FIRST_DAY|VALUE_TYPE|VALUE_NAME|REPORT_STATUS|VALUE|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = @source_type,
    @location_type = @location_type,
    @delimiter_type = @delimiter_type,
    @security_profile= @security_profile;



#### Stage to inter

In [ ]:
-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);

SET @source = '[dbo].[SP_VWS_VACCINE_DELIVERIES_ADMINISTERED_INTER]';
SET @source_name = 'SOURCE_Inter_VWS_Vaccination_Deliveries_Administered';
SET @source_description = 'Moves VWS Vaccination Deliveries Administered data from stage table to intermediate table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

#### Inter to dest

In [ ]:

-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = '[dbo].[SP_VWS_VACCINE_ADMINISTERED_DEST]';
SET @source_name = 'SOURCE_Dest_VWS_Vaccination_Administered';
SET @source_description = 'Moves VWS Vaccination Administered data from intermediate table to dest table'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = '[dbo].[SP_VWS_VACCINE_DELIVERIES_DEST]';
SET @source_name = 'SOURCE_Dest_VWS_Vaccination_Deliveries';
SET @source_description = 'Moves VWS Vaccination Deliveries data from intermediate table to dest table'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = '[dbo].[SP_VWS_VACCINE_DELIVERIES_WEEKLY]';
SET @source_name = 'SOURCE_Dest_VWS_Vaccination_Deliveries_Weekly';
SET @source_description = 'Moves VWS Vaccination Weekly Deliveries data from intermediate table to dest table'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

In [ ]:
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256);
        
SET @source = 'DBO.SP_VWS_VACCINE_PLANNED_DEST';
SET @source_name = 'SOURCE_SP_VWS_VACCINE_PLANNED_DEST';
SET @source_description = 'Makes sure that the loading of data into the tables goes without problems'; 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

### Processes

In [ ]:
-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256),
        @workflow_name NVARCHAR(50) = 'VWS_VACCINE_DELIVERIES_ADMINISTERED';

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_Load_VWS_Vaccination_Deliveries_Administered';
SET @process_description =  'Toegediende vaccinaties';
SET @process_source_name = 'SOURCE_Load_VWS_Vaccination_Deliveries_Administered';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @process_description =  'Moves VWS Vaccination Deliveries Administered data from stage table to intermediate table';
SET @process_source_name = 'SOURCE_Inter_VWS_Vaccination_Deliveries_Administered';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
SET @process_name = 'PROCESS_Dest_VWS_Vaccination_Administered';
SET @process_description =  'Moves VWS Vaccination Administered data from intermediate table to dest table';
SET @process_source_name = 'SOURCE_Dest_VWS_Vaccination_Administered';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;


SET @process_name = 'PROCESS_Dest_VWS_Vaccination_Deliveries';
SET @process_description =  'Moves VWS Vaccination Deliveries data from intermediate table to dest table';
SET @process_source_name = 'SOURCE_Dest_VWS_Vaccination_Deliveries';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

SET @process_name = 'PROCESS_Dest_VWS_Vaccination_Deliveries_Weekly';
SET @process_description =  'Moves VWS Vaccination Weekly Deliveries data from intermediate table to dest table';
SET @process_source_name = 'SOURCE_Dest_VWS_Vaccination_Deliveries_Weekly';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 0;

SET @process_name = 'PROCESS_Dest_VWS_Vaccine_Planned';
SET @process_description =  'Inserts the planned number of shot for the given week';
SET @process_source_name = 'SOURCE_SP_VWS_VACCINE_PLANNED_DEST';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;



### Dependencies

In [ ]:
-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256),
        @workflow_name NVARCHAR(50) = 'VWS_VACCINE_DELIVERIES_ADMINISTERED';

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_dataflow_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_process_name = 'PROCESS_Load_VWS_Vaccination_Deliveries_Administered';
SET @dependency_description = 'Dependency for PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;




-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_VWS_Vaccination_Deliveries';
SET @dependency_dataflow_name ='PROCESS_Dest_VWS_Vaccination_Deliveries';
SET @dependency_process_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_description = 'Dependency for PROCESS_Dest_VWS_Vaccination_Deliveries' ;

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_VWS_Vaccination_Administered';
SET @dependency_dataflow_name = 'PROCESS_Dest_VWS_Vaccination_Administered';
SET @dependency_process_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_description = 'Dependency for PROCESS_Dest_VWS_Vaccination_Administered';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_VWS_Vaccination_Deliveries_Weekly';
SET @dependency_dataflow_name ='PROCESS_Dest_VWS_Vaccination_Deliveries_Weekly';
SET @dependency_process_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_description = 'Dependency for PROCESS_Dest_VWS_Vaccination_Deliveries_Weekly';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 0;


-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_VWS_Vaccine_Planned';
SET @dependency_dataflow_name = 'PROCESS_Dest_VWS_Vaccine_Planned';
SET @dependency_process_name = 'PROCESS_Inter_VWS_Vaccination_Deliveries_Administered';
SET @dependency_description = 'Dependency for PROCESS_Dest_VWS_Vaccine_Planned';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_ADMINISTERED_PLANNED_TEMP',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_planned',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_END_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_ADMINISTERED',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_ADMINISTERED_EST',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_administered_estimate',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_DELIVERIES',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_delivery',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_DELIVERIES_EST',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_delivery_estimate',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_DELIVERIES_WEEKLY',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_delivery_per_supplier',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_VWS_VACCINE_DELIVERIES_WEEKLY_TIMESPAN',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_delivery_estimate_time_span',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO